# Aprendizaje Automático - Intermedio

## Variables categóricas

### Introducción

Aprenderemos qué es una variable categórica a través de tres enfoques para manejar este tipo de datos.

Una **variable categórica** toma solo un número limitado de valores.

+ Considera una encuesta que te pregunta con qué frecuencia desayunas y te ofrece cuatro opciones: "Nunca", "Raramente", "La mayoría de los días" o "Todos los días". En este caso, los datos son categóricos porque las respuestas se dividen en un conjunto fijo de categorías.
+ Si las personas respondieran a una encuesta sobre qué marca de automóvil poseen, las respuestas estarían en categorías como "Honda", "Toyota" y "Ford". En este caso, los datos también son categóricos.

Obtendremos un error si intentamos utilizar estas variables en la mayoría de los modelos de aprendizaje automático en Python sin preprocesarlas primero. En este tutorial, compararemos tres enfoques que podemos usar para preparar nuestros datos categóricos.

### Tres Enfoques

#### 1) Eliminar las variables categóricas

El enfoque más fácil para tratar con variables categóricas es simplemente eliminarlas del conjunto de datos. Este enfoque solo funcionará bien si las columnas no contienen información útil.

#### 2) Label Encoding (Codificación de Etiquetas)

**Label encoding** asigna cada valor único a un número entero diferente.

![label_encoding](./images/label_encoding.png)

Este enfoque supone un orden de las categorías: "Nunca" (0) < "Raramente" (1) < "La mayoría de los días" (2) < "Todos los días" (3).

Esta suposición tiene sentido en este ejemplo, porque hay una clasificación indiscutible para las categorías. No todas las variables categóricas tienen un orden claro en los valores, pero nos referimos a las que lo hacen como **variables ordinales**. Para los modelos basados en árboles (como árboles de decisión y random forests), podemos esperar que label encoding funcione bien con variables ordinales.

#### 3) One-Hot Encoding

**One-Hot encoding** crea nuevas columnas que indican la presencia (o ausencia) de cada valor posible en los datos originales. Para entender esto, trabajaremos con un ejemplo.

![one_hot_encoding](./images/one_hot_encoding.png)

En el conjunto de datos original, "Color" es una variable categórica con tres categorías: "Rojo", "Amarillo" y "Verde". La codificación one-hot correspondiente contiene una columna para cada valor posible y una fila para cada fila en el conjunto de datos original. Dondequiera que el valor original fuera "Rojo", colocamos un 1 en la columna "Rojo"; si el valor original era "Amarillo", colocamos un 1 en la columna "Amarillo", y así sucesivamente.

A diferencia de label encoding, la codificación one-hot no asume un orden de las categorías. Por lo tanto, podemos esperar que este enfoque funcione particularmente bien si no hay un orden claro en los datos categóricos (por ejemplo, "Rojo" no es ni más ni menos que "Amarillo"). Nos referimos a las variables categóricas sin una clasificación intrínseca como **variables nominales**.

La codificación one-hot generalmente no funciona bien si la variable categórica tiene una gran cantidad de valores (es decir, generalmente no la usará para variables que toman más de 15 valores diferentes).

### Ejemplo

En el ejemplo, trabajaremos con el conjunto de datos de [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot).

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lee los datos
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Separa el objetivo de los predictores
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide los datos en subconjuntos de entrenamiento y validación
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Elimina las columnas con valores ausentes (enfoque más simple)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinalidad" significa el número de valores únicos en una columna
# Selecciona las columnas categóricas con relativamente baja cardinalidad (conveniente pero arbitrariamente)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Selecciona las columnas numéricas
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Mantiene solo las columnas seleccionadas
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

c:\program files\python37\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Echamos un vistazo a los datos de entrenamiento.

In [9]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


A continuación, obtenemos una lista de todas las variables categóricas en los datos de entrenamiento.

Hacemos esto comprobando el tipo de datos (o **dtype**) de cada columna. El objeto `dtype` indica que una columna tiene texto (hay otras cosas que en teoría podría ser, pero eso no es importante para nuestros propósitos). Para este conjunto de datos, las columnas con texto indican variables categóricas.

In [10]:
# Obtiene la lista de variables categóricas
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Variables categóricas:")
print(object_cols)

Variables categóricas:
['Type', 'Method', 'Regionname']


#### Definir la función para medir la calidad de cada enfoque

Definimos una función `score_dataset()` para comparar los tres enfoques diferentes para tratar con variables categóricas. Esta función informa el [error absoluto medio](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) de un modelo random forest. En general, ¡queremos que el MAE sea lo más bajo posible!

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Puntuación del Enfoque 1 (Eliminar variables categóricas)
Eliminamos las columnas `object` con el método `select_dtypes()`.

In [12]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE de Enfoque 1 (Eliminar variables categóricas):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE de Enfoque 1 (Eliminar variables categóricas):
175703.48185157913


#### Puntuación del Enfoque 2 (Label Encoding)

Scikit-learn tiene una clase `LabelEncoder` que se puede usar para obtener label encodings. Recorremos las variables categóricas y aplicamos el label encoder por separado a cada columna.

In [13]:
from sklearn.preprocessing import LabelEncoder

# Hace una copia para evitar cambiar los datos originales
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Aplica label encoder a cada columna con datos categóricos
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE de Enfoque 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE de Enfoque 2 (Label Encoding):
165936.40548390493


En el código anterior, para cada columna asignamos aleatoriamente cada valor único a un entero diferente. Este es un enfoque común que es más simple que proporcionar etiquetas personalizadas; sin embargo, podemos esperar un aumento adicional en el rendimiento si proporcionamos etiquetas mejor informadas para todas las variables ordinales.

#### Puntuación del Enfoque 3 (One-Hot Encoding)

Usamos la clase `OneHotEncoder` de scikit-learn para obtener codificaciones one-hot. Hay varios parámetros que se pueden usar para personalizar su comportamiento.

+ Establecemos `handle_unknown = 'ignore'` para evitar errores cuando los datos de validación contienen clases que no están representadas en los datos de entrenamiento, y
+ establecer `sparse = False` asegura que las columnas codificadas se devuelvan como una matriz numpy (en lugar de una matriz dispersa).

Para usar el codificador, suministramos solo las columnas categóricas que queremos que estén codificadas one-hot. Por ejemplo, para codificar los datos de entrenamiento, proporcionamos `X_train[object_cols]`. (object_cols en el código a continuación es una lista de los nombres de columna con datos categóricos, por lo que `X_train[object_cols]` contiene todos los datos categóricos en el conjunto de entrenamiento).

In [14]:
from sklearn.preprocessing import OneHotEncoder

# Aplica one-hot encoder a cada columna con datos categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding elimina el índica; lo recuperamos
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Elimina columnas categóricas (las reemplazará con one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Añade columnas one-hot a características numéricas
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE de Enfoque 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE de Enfoque 3 (One-Hot Encoding):
166089.4893009678


### ¿Qué enfoque es el mejor?

En este caso, eliminar las columnas categóricas (**Enfoque 1**) tuvo el peor desempeño, ya que tenía una valor MAE más alto. En cuanto a los otros dos enfoques, dado que los valores MAE devueltos tienen un valor tan cercano, no parece haber ningún beneficio significativo de uno sobre el otro.

En general, one-hot encoding (**Enfoque 3**) funcionará mejor habitualmente y eliminar las columnas categóricas (**Enfoque 1**) generalmente funcionará peor, pero varía según el caso.

### Conclusión

El mundo está lleno de datos categóricos. ¡Serás un científico de datos mucho más efectivo si sabes cómo usar este tipo de datos tan común!

## Ejercicio

### Configuración

En este ejercicio trabajaremos con datos de [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Vamos a cargar los conjuntos de entrenamiento y validación en `X_train`, `X_valid`, `y_train`, e `y_valid`.  El conjunto de pruebas es cargado en `X_test`.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lee los datos
X = pd.read_csv('./input/train.csv', index_col='Id')
X_test = pd.read_csv('./input/test.csv', index_col='Id')

# Elimina las filas con objetivos ausentes, separa el objetivo de los predictores
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Para mantener las cosas simples solo usaremos predictores numéricos
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Separamos los datos de validación a partir de los datos de entrenamiento
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [16]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


Observa que el conjunto de datos contiene variables numéricas y categóricas. Tendrás que codificar los datos categóricos antes de entrenar un modelo.

Para comparar diferentes modelos, usarás la misma función `score_dataset()` del tutorial. Esta función informa el [error absoluto medio](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) de un modelo random forest.

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Paso 1: eliminar columnas con datos categóricos

Comenzarás con el enfoque más directo. Vamos a preprocesar los datos en `X_train` y `X_valid` para eliminar columnas con datos categóricos. Establece los DataFrames preprocesados en `drop_X_train` y `drop_X_valid`, respectivamente.

In [18]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [19]:
print("MAE Enfoque 1 (Eliminar variables categóricas):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE Enfoque 1 (Eliminar variables categóricas):
17837.82570776256


### Paso 2: Label encoding

Antes de saltar a label encoding, investigaremos el conjunto de datos. Específicamente, veremos la columna `Condition2`. El siguiente código imprime las entradas únicas en los conjuntos de entrenamiento y validación.

In [20]:
print("Valores únicos en la columna 'Condition2' de los datos de entrenamiento:", X_train['Condition2'].unique())
print("\nValores únicos en la columna 'Condition2' de los datos de validación:", X_valid['Condition2'].unique())

Valores únicos en la columna 'Condition2' de los datos de entrenamiento: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Valores únicos en la columna 'Condition2' de los datos de validación: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


Si ahora escribes código para:
- entrenar un label encoder a los datos de entrenamiento, y luego
- lo usas para transformar los datos de entrenamiento y validación,

obtendrás un error. ¿Puedes ver por qué esto es así?

**Solución**: Al entrenar un label encoder a una columna en los datos de entrenamiento se crea una etiqueta de valor entero para cada valor único **que aparece en los datos de entrenamiento**. En el caso de que los datos de validación contengan valores que no aparecen también en los datos de entrenamiento, el codificador arrojará un error, porque estos valores no tendrán un número entero asignado. Observe que la columna `Condition2` en los datos de validación contiene los valores `RRAn` y `RRNn`, pero estos no aparecen en los datos de entrenamiento; por lo tanto, si intentamos usar un label encoder con scikit-learn, el código arrojará un error.

Este es un problema común que encontrarás con los datos del mundo real y hay muchos enfoques para solucionar este problema. Por ejemplo, puedes escribir un label encoder personalizado para tratar con nuevas categorías. Sin embargo, el enfoque más simple es eliminar las columnas categóricas problemáticas.

Vamos a guardar las columnas problemáticas en una lista de Python `bad_label_cols`. Del mismo modo, las columnas que pueden codificarse de forma segura se almacenan en `good_label_cols`.

In [21]:
# Todas las columnas categóricas
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columnas que pueden ser satisfactoriamente label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Columnas problemáticas que serán eliminadas del dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Columnas categóricas que serán label encoded:', good_label_cols)
print('\nColulmnas categóricas que serán eliinadas del dataset:', bad_label_cols)

Columnas categóricas que serán label encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']

Colulmnas categóricas que serán eliinadas del dataset: ['SaleType', 'RoofStyle', 'ExterCond', 'Condition1', 'RoofMatl', 'Condition2', 'Neighborhood', 'Heating', 'Utilities', 'Exterior1st', 'Functional', 'HeatingQC', 'Exterior2nd', 'Foundation', 'LandSlope']


Vamos a label encode los datos en `X_train` y `X_valid`. Establece los DataFrames preprocesados en `label_X_train` y `label_X_valid`, respectivamente.
- Hemos proporcionado el siguiente código para eliminar las columnas categóricas en `bad_label_cols` del conjunto de datos.
- Debe label encode las columnas categóricas en `good_label_cols`.

In [22]:
from sklearn.preprocessing import LabelEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()
for col in good_label_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

In [23]:
print("MAE Enfoque 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE Enfoque 2 (Label Encoding):
17575.291883561644


### Paso 3 Investigando la cardinalidad

Hasta ahora has probado dos enfoques diferentes para tratar con variables categóricas. Y has visto que codificar datos categóricos produce mejores resultados que eliminar columnas del conjunto de datos.

Pronto intentará la codificación one-hot. Antes de eso hay un tema adicional que debemos cubrir.

In [24]:
# Obtiene el número de entradas únicas en cada columns con datos categóricos
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Muestra el número de entradas únicas por columna, en orden ascendente
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

El resultado anterior muestra, para cada columna con datos categóricos, el número de valores únicos. Por ejemplo, la columna `Street` en los datos de entrenamiento tiene dos valores únicos: `Grvl` y `Pave`, correspondientes a un camino de grava y un camino pavimentado, respectivamente.

Nos referimos al número de entradas únicas de una variable categórica como la **cardinalidad** de esa variable categórica. Por ejemplo, la variable `Street` tiene cardinalidad 2.

*¿Cuantas variables categóricas en los datos de entrenamiento tiene cardinalidad mayor de 10?**

In [25]:
high_cardinality_numcols = 3

**¿Cuántas columnas son necesarias para codificar one-hot la variable `Neighborhood` en los datos de entrenamiento?**

In [26]:
num_cols_neighborhood = 25

Para conjuntos de datos grandes con muchas filas, la codificación one-hot puede expandir en gran medida el tamaño del conjunto de datos. Por esta razón, normalmente solo codificaremos one-hot columnas con una cardinalidad relativamente baja. Luego las columnas de alta cardinalidad pueden eliminarse del conjunto de datos o podemos usar label encoding.

Como ejemplo, considere un conjunto de datos con 10.000 filas y que contenga una columna categórica con 100 entradas únicas.
- Si esta columna se reemplaza con la codificación one-hot, ¿cuántas entradas se agregan al conjunto de datos?
- Si en su lugar reemplazamos la columna con label encoding, ¿cuántas entradas se agregan?

In [29]:
OH_entries_added = (10000 * 100) - 10000
label_entries_added = 0